In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from google.colab import files

In [ ]:
df = pd.read_csv('Online_Retail.csv')

In [ ]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   541909 non-null  int64  
 1   InvoiceNo    541909 non-null  object 
 2   StockCode    541909 non-null  object 
 3   Description  540455 non-null  object 
 4   Quantity     531003 non-null  float64
 5   InvoiceDate  536460 non-null  object 
 6   UnitPrice    525937 non-null  float64
 7   CustomerID   406829 non-null  float64
 8   Country      498234 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 37.2+ MB


,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,NaN
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,NaN


In [ ]:
df.describe()

,Unnamed: 0,Quantity,UnitPrice,CustomerID
count,541909.00000,531003.000000,525937.000000,406829.000000
mean,270954.00000,9.535347,4.636641,15287.690570
std,156435.79785,220.146728,98.163901,1713.600303
min,0.00000,-80995.000000,-11062.060000,12346.000000
25%,135477.00000,1.000000,1.250000,13953.000000
50%,270954.00000,3.000000,2.080000,15152.000000
75%,406431.00000,10.000000,4.130000,16791.000000
max,541908.00000,80995.000000,38970.000000,18287.000000


In [ ]:
df[['InvoiceNo', 'StockCode', 'Description', 'Country']].describe(include='object')

,InvoiceNo,StockCode,Description,Country
count,541909,541909,540455,498234
unique,25900,4070,4223,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,1114,2313,2369,455567


In [ ]:
df.shape

(541909, 9)

In [ ]:
# Create CustomerID to Country mapping (first non-null country per customer)
customer_countries = df.dropna().drop_duplicates('CustomerID').set_index('CustomerID')['Country']

# Fill missing countries using the mapping
df['Country'] = df['Country'].fillna(df['CustomerID'].map(customer_countries))

In [ ]:
# Calculate average price per StockCode
avg_prices = df.groupby('StockCode')['UnitPrice'].mean()

# Fill missing prices using the averages
df['UnitPrice'] = df['UnitPrice'].fillna(df['StockCode'].map(avg_prices))

In [ ]:
df = df.dropna(subset=['CustomerID'])

In [ ]:
df = df.dropna(subset=['InvoiceDate'])

In [ ]:
df = df.dropna(subset=['Quantity'])

In [ ]:
df['Description'] = df['Description'].astype(str)
bad_description= (
    df['Description'].str.startswith('?') |
    df['Description'].str.startswith('*') |
    df['Description'].str.contains('20713', regex=False)
)

bad_count = bad_description.sum()
df = df[~bad_description]

In [ ]:
invalid_countries = ['European Community', 'Unspecified']
df = df[~df['Country'].isin(invalid_countries)]

In [ ]:
renameCountry = {'EIRE': 'Ireland', 'RSA': 'South Africa', 'USA': 'United States of America'}
df['Country'] = df['Country'].replace(renameCountry)

In [ ]:
df = df[df['Description'] != 'POSTAGE']

In [ ]:
dropStockCode = ['BANK CHARGES', 'C2', 'CRUK', 'D', 'DOT', 'M', 'PADS']
df = df[~df['StockCode'].isin(dropStockCode)]

In [ ]:
#drop remaining null values
df = df.dropna(subset=['UnitPrice', 'Country'])

In [ ]:
df = df[df['Quantity'] > 0]

In [ ]:
df = df[df['UnitPrice'] > 0.00]

In [ ]:
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 384109 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    384109 non-null  object 
 1   StockCode    384109 non-null  object 
 2   Description  384109 non-null  object 
 3   Quantity     384109 non-null  float64
 4   InvoiceDate  384109 non-null  object 
 5   UnitPrice    384109 non-null  float64
 6   CustomerID   384109 non-null  float64
 7   Country      384109 non-null  object 
dtypes: float64(3), object(5)
memory usage: 26.4+ MB


In [ ]:
print(df['Country'].unique())

['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'Ireland' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Finland' 'Austria' 'Greece' 'Singapore' 'Lebanon'
 'United Arab Emirates' 'Israel' 'Saudi Arabia' 'Czech Republic' 'Canada'
 'Brazil' 'United States of America' 'Bahrain' 'Malta' 'South Africa']


In [ ]:
country_encoded = pd.get_dummies(df['Country'], prefix='Country')

In [ ]:
df_encoded = pd.concat([df, country_encoded], axis=1)

# Drop the original 'Country' column (optional)
df_encoded = df_encoded.drop('Country', axis=1)

In [ ]:
print("\nDataFrame after one-hot encoding:")
print(df_encoded.head())


DataFrame after one-hot encoding:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER       6.0   
1    536365     71053                  WHITE METAL LANTERN       6.0   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER       8.0   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE       6.0   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.       6.0   

           InvoiceDate  UnitPrice  CustomerID  Country_Australia  \
0  2010-12-01 08:26:00       2.55     17850.0              False   
1  2010-12-01 08:26:00       3.39     17850.0              False   
2  2010-12-01 08:26:00       2.75     17850.0              False   
3  2010-12-01 08:26:00       3.39     17850.0              False   
4  2010-12-01 08:26:00       3.39     17850.0              False   

   Country_Austria  Country_Bahrain  ...  Country_Portugal  \
0            False            False  ...             False   

In [ ]:
df_encoded.to_csv('encoded_online_retail.csv', index=False)
files.download('encoded_online_retail.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save the DataFrame as CSV
df.to_csv("cleaned_online_retail.csv", index=False)

# Download the CSV
files.download("cleaned_online_retail.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>